In [32]:
import cv2
import os

# Global variable to store the click position
click_position = None
frame_copy = None

# Callback function to handle mouse click events
def on_click(event, x, y, flags, param):
    global click_position, frame_copy
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        click_position = (x, y)
        print(f"Clicked at: ({x}, {y})")
        
        # Draw a red cross at the clicked position
        # Create a copy of the frame to draw the cross
        frame_copy = frame.copy()
        cv2.line(frame_copy, (x-20, y), (x+20, y), (0, 0, 255), 2)  # Horizontal line
        cv2.line(frame_copy, (x, y-20), (x, y+20), (0, 0, 255), 2)  # Vertical line

# Function to save the last click position to a text file
def save_click_position(video_path, position):
    if position:
        # Create the text file path by changing extension to .txt
        txt_file_path = os.path.splitext(video_path)[0] + "_reward_position.txt"
        with open(txt_file_path, 'w') as f:
            f.write(f"Reward Position: x={position[0]}, y={position[1]}")
        print(f"Saved reward position to {txt_file_path}")

# Path to the root folder containing video files
video_folder = '//10.69.168.1/crnldata/forgetting/Aurelie/CheeseboardExperiment/DAQ_data/AB/Test'

# Walk through the directory recursively
for root, dirs, files in os.walk(video_folder):
    # Filter for video files (avi, mp4, mov)
    video_files = [f for f in files if f.endswith(('.avi', '.mp4', '.mov'))]
    
    # Process only videos where path contains "MyFirstWebcam"
    for video_file in video_files:
        video_path = os.path.join(root, video_file)
        # Normalize the path to avoid mixed slashes
        video_path = os.path.normpath(video_path)
        
        if "My_First_WebCam" in video_path and '\\Cheeseboard\\' in video_path and 'DLC' not in video_file:
            
            # Load the video file
            cap = cv2.VideoCapture(video_path)

            # Check if the video file is opened
            if not cap.isOpened():
                print(f"Error: Could not open video {video_file}")
                continue

            # Read the first frame
            ret, frame = cap.read()
            if not ret:
                print(f"Error: Could not read the first frame of {video_file}")
                cap.release()
                continue

            # Create a copy of the frame to update it after click
            frame_copy = frame.copy()

            # Display the first frame using OpenCV
            cv2.imshow(video_path, frame_copy)

            # Set up the callback for mouse clicks
            cv2.setMouseCallback(video_path, on_click)

            # Wait for a key press (user can click on the frame)
            while True:
                # Display the updated frame (with red cross if clicked)
                cv2.imshow(video_path, frame_copy)
                
                # Wait for a key press
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):  # Press 'q' to quit
                    break

            # Save the click position to a text file before quitting
            if click_position:
                save_click_position(video_path, click_position)

            # Release the video and close the window
            cap.release()
            cv2.destroyAllWindows()

print("Processing completed for all relevant videos.")


Clicked at: (362, 318)
Saved reward position to \\10.69.168.1\crnldata\forgetting\Aurelie\CheeseboardExperiment\DAQ_data\AB\Test\Lou\Cheeseboard\2024_12_06\11_07_47\My_First_WebCam\0_compressed_reward_position.txt
Clicked at: (362, 318)
Saved reward position to \\10.69.168.1\crnldata\forgetting\Aurelie\CheeseboardExperiment\DAQ_data\AB\Test\Blue\Cheeseboard\2024_12_06\14_52_03\My_First_WebCam\0_compressed_reward_position.txt
Clicked at: (362, 318)
Saved reward position to \\10.69.168.1\crnldata\forgetting\Aurelie\CheeseboardExperiment\DAQ_data\AB\Test\Green\Cheeseboard\2024_12_06\17_17_38\My_First_WebCam\0_compressed_reward_position.txt
Processing completed for all relevant videos.
